<a href="https://colab.research.google.com/github/sw6820/swm_prototype/blob/main/oddiya_langchain_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================================================
#  CELL 1: SETUP - INSTALL LIBRARIES
# =======================================================================
!pip install -qU langchain langchain-google-genai>=0.11.0 googlemaps google-generativeai>=0.8.5

print("✅ Libraries installed. You can now run the next cell.")

✅ Libraries installed. You can now run the next cell.


In [ ]:
# =======================================================================
#  CELL 2: IMPORTS & API KEY CONFIGURATION
# =======================================================================
import os
from google.colab import userdata

# LangChain specific imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# --- API Key Configuration ---
try:
    # Get Gemini API Key
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("✅ Google Gemini API Key configured successfully.")
except Exception as e:
    print(f"🚨 SECRET NOT FOUND or other error: {e}. Please ensure 'GOOGLE_API_KEY' is set in Colab Secrets.")

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


✅ Google Gemini API Key configured successfully.


In [ ]:
# =======================================================================
#  CELL 3: BACKEND - REVISED DATA STRUCTURES
# =======================================================================
# REVISION: The descriptions now clarify that only the VALUE should be in Korean.

class GenerativeActivity(BaseModel):
    name: str = Field(description="The name of the recommended place. The value for this key MUST be in Korean (한국어).")
    description: str = Field(description="A brief description of the activity. The value for this key MUST be in Korean (한국어).")

class DailyGenerativePlan(BaseModel):
    day_number: int = Field(description="The day number for this plan (e.g., 1, 2).")
    city: str = Field(description="The city for this plan. The value for this key MUST be in Korean (한국어).")
    theme_of_the_day: str = Field(description="A fun theme for the day. The value for this key MUST be in Korean (한국어).")
    morning_activity: GenerativeActivity = Field(description="The morning activity. Its content must be in Korean.")
    afternoon_activity: GenerativeActivity = Field(description="The afternoon activity. Its content must be in Korean.")

class FullTripPlan(BaseModel):
    """The complete, structured set of generated recommendations for the trip."""
    plans: List[DailyGenerativePlan]

# This object will automatically parse the LLM's raw text output.
parser = PydanticOutputParser(pydantic_object=FullTripPlan)

print("✅ 백엔드 데이터 구조 및 파서 업데이트 완료 (키/값 언어 분리).")

✅ 백엔드 데이터 구조 및 파서 업데이트 완료 (키/값 언어 분리).


In [ ]:
# =======================================================================
#  CELL 4: BACKEND - HIGH-PRECISION LANGCHAIN PROMPT
# =======================================================================
# REVISION: The prompt now makes a critical distinction between JSON keys and values.

generative_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a travel planner for families. Your task is to generate a JSON object that strictly follows the provided schema. "
               "It is critical that you follow these two rules:\n"
               "1. The JSON 'keys' (like 'plans', 'day_number', 'city', 'theme_of_the_day', 'morning_activity', 'afternoon_activity', 'name', 'description') MUST be in English, exactly as provided in the schema.\n"
               "2. The JSON 'values' (the actual content like the theme, descriptions, and place names) MUST be written in Korean (한국어).\n"
               "Do not translate the keys. Only translate the content. Ensure the JSON output strictly adheres to the FullTripPlan schema, using only the specified keys and nested structure. Do NOT include extraneous keys like 'plan_details' or 'time'."),
    ("human", "Please generate a complete {num_days}-day trip itinerary for {city}. "
              "Remember: Keep the JSON structure and keys in English, but write all the descriptive text content in Korean (한국어).\n"
              "{format_instructions}")
])

# Initialize the Gemini model. A slightly lower temperature can help with following rules.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)

# Chain the prompt, model, and parser together.
generative_chain = generative_prompt_template | llm | parser

print("✅ 생성형 추천 체인 정밀도 향상 완료.")

✅ 생성형 추천 체인 정밀도 향상 완료.


# =======================================================================
#  CELL 5: DEFINE INPUT & RUN THE ENGINE
# =======================================================================

# --- 여기에 입력을 수정하세요 ---
trip_plan = {
    "서울": 2,
    "부산": 2
}
# --------------------------------

print("===================================================")
print(" G.O.A.T. 생성형 엔진 (콘솔 모드) ")
print("===================================================")
print(f"\n여행 계획: {trip_plan}")

all_plans = []
full_day_counter = 1

# Iterate through the user-defined trip plan
for city, num_days in trip_plan.items():
    print(f"\n[엔진] {city}를 위한 {num_days}일 계획을 Gemini에게 요청 중...")

    try:
        # Prepare the input for the LangChain chain
        chain_input = {
            "city": city,
            "num_days": num_days,
            "format_instructions": parser.get_format_instructions(),
        }
        # Invoke the chain to get the structured result
        result = generative_chain.invoke(chain_input)

        # Process and number the days sequentially for the whole trip
        for plan in result.plans:
            plan.day_number = full_day_counter
            all_plans.append(plan)
            full_day_counter += 1

    except Exception as e:
        print(f"[오류] {city} 추천 생성 중 오류 발생: {e}")

# --- 최종 결과를 콘솔에 출력 ---
print("\n\n===================================================")
print("          ✨ 최종 생성된 여행 계획 ✨")
print("===================================================")

if not all_plans:
    print("\n생성된 계획이 없습니다.")
else:
    for day_plan in all_plans:
        print("\n---------------------------------------------------")
        print(f"  {day_plan.day_number}일차: {day_plan.city}에서의 '{day_plan.theme_of_the_day}'")
        print("---------------------------------------------------")

        morning = day_plan.morning_activity
        print(f"\n  🌅 오전: {morning.name}")
        print(f"  👉 {morning.description}")

        afternoon = day_plan.afternoon_activity
        print(f"\n  ☀️ 오후: {afternoon.name}")
        print(f"  👉 {afternoon.description}")

print("\n\n===================================================")
print("           [엔진 실행 완료]")
print("===================================================")